In [1]:
import findspark
from pyspark.sql import SparkSession

In [2]:
findspark.init()

In [3]:
spark = SparkSession.builder.appName("flight2015").getOrCreate()

23/11/11 00:33:40 WARN Utils: Your hostname, bagjunhyeog-ui-noteubug.local resolves to a loopback address: 127.0.0.1; using 172.30.1.11 instead (on interface en0)
23/11/11 00:33:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 00:33:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.setLogLevel("error")

In [5]:
flightData2015 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("./data/flight-data/csv/2015-summary.csv")

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [7]:
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

## `DataFrame`과 `SQL`

In [8]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [9]:
sqlWay = spark.sql("""
SELECT dest_country_name, count(1)
FROM flight_data_2015
GROUP BY dest_country_name
""")

In [12]:
dataFrameWay = flightData2015.groupby("DEST_COUNTRY_NAME").count()

In [13]:
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[dest_country_name#17], functions=[count(1)])
   +- Exchange hashpartitioning(dest_country_name#17, 5), ENSURE_REQUIREMENTS, [plan_id=39]
      +- HashAggregate(keys=[dest_country_name#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/hotamul/SparkProjects/MovieRating/spark-the-definitive-gui..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 5), ENSURE_REQUIREMENTS, [plan_id=52]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemo

In [14]:
from pyspark.sql import functions as func

In [15]:
flightData2015.select(func.max("count")).take(1)

[Row(max(count)=370002)]

In [17]:
maxSql = spark.sql("""
SELECT dest_country_name, sum(count) as destination_total
FROM flight_data_2015
GROUP BY dest_country_name
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|dest_country_name|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [20]:
flightData2015.groupBy("dest_country_name")\
    .sum("count")\
    .withColumnRenamed("sum(count)", "destination_total")\
    .sort(func.desc("destination_total"))\
    .limit(5)\
    .show()

+-----------------+-----------------+
|dest_country_name|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [21]:
flightData2015.groupBy("dest_country_name")\
    .sum("count")\
    .withColumnRenamed("sum(count)", "destination_total")\
    .sort(func.desc("destination_total"))\
    .limit(5)\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#109L DESC NULLS LAST], output=[dest_country_name#17,destination_total#109L])
   +- HashAggregate(keys=[dest_country_name#17], functions=[sum(count#19)])
      +- Exchange hashpartitioning(dest_country_name#17, 5), ENSURE_REQUIREMENTS, [plan_id=190]
         +- HashAggregate(keys=[dest_country_name#17], functions=[partial_sum(count#19)])
            +- FileScan csv [DEST_COUNTRY_NAME#17,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/hotamul/SparkProjects/MovieRating/spark-the-definitive-gui..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [22]:
spark.stop()